In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf



In [2]:
import sys
sys.path.insert(1, '../Pre Processing Stage/')
sys.path.insert(1, '../Processing Stage/HoG')


from commonfunctions import *
from main import *
from hog import *

In [3]:
final_lbp_flattened=np.load('./numpyArrays/final_lbp_flattened.npy')
targets_lbp=np.load('./numpyArrays/targets_lbp.npy')



In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from tensorflow import keras

In [5]:
classifiers1 = {
    'SVM': SVC(random_state=40,kernel='poly'),
   
    
}

In [6]:
classifiers2 = {
    'KNN': KNeighborsClassifier(n_neighbors=7),
    
    
}

In [7]:
classifiers3 = {
    
    'rnd': RandomForestClassifier(random_state=40),
   
    
}

In [8]:
classifiers4 = {
    
    'voting_clf_soft' : VotingClassifier(
    estimators=[
        ('KNN', KNeighborsClassifier(n_neighbors=7)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42,probability=True))
    ],
    voting='soft'), 
   
    
}

In [9]:
classifiers5 = {
  
    'voting_clf_hard': VotingClassifier(
    estimators=[
            ('KNN', KNeighborsClassifier(n_neighbors=7)),
            ('rf', RandomForestClassifier(random_state=42)),
            ('svc', SVC(random_state=42))
        ],
    voting='hard'),
    
}

In [10]:
from sklearn.metrics import confusion_matrix
import copy


def train_model(x,y,testSize,classifiers):
    models=[]

    X_train, X_test, y_train, y_test=train_test_split(x, y,test_size=testSize,stratify=y)

    for model_name, model in copy.deepcopy(classifiers).items():
        
        print('############## Training', model_name, "##############")
        model.fit(X_train, y_train)

        accuracy = model.score(X_test, y_test)

        models.append({model_name:model})

        print(model_name, 'accuracy:', accuracy*100, '%')
        y_pred = model.predict(X_test)

        mat = confusion_matrix(y_test, y_pred)

        print(mat)
            

    return models

In [11]:
models_lbp = train_model(final_lbp_flattened, targets_lbp, 0.2,classifiers1)

############## Training SVM ##############


SVM accuracy: 100.0 %
[[17  0]
 [ 0 21]]


In [12]:
models_lbp2 = train_model(final_lbp_flattened, targets_lbp, 0.2,classifiers2)

############## Training KNN ##############
KNN accuracy: 100.0 %
[[17  0]
 [ 0 21]]


In [13]:
models_lbp3 = train_model(final_lbp_flattened, targets_lbp, 0.2,classifiers3)

############## Training rnd ##############
rnd accuracy: 94.73684210526315 %
[[16  1]
 [ 1 20]]


In [14]:
models_lbp4 = train_model(final_lbp_flattened, targets_lbp, 0.2,classifiers4)

############## Training voting_clf_soft ##############
voting_clf_soft accuracy: 100.0 %
[[17  0]
 [ 0 21]]


In [15]:
models_lbp5 = train_model(final_lbp_flattened, targets_lbp, 0.2,classifiers5)

############## Training voting_clf_hard ##############
voting_clf_hard accuracy: 97.36842105263158 %
[[17  0]
 [ 1 20]]


In [22]:
import pickle

def save_models(modelsArray,suffix):
    for model in modelsArray:
        pickle.dump(next(iter(model.values())), open(
            f"./models/{next(iter(model.keys()))}_{suffix}.pkl", "wb"))

In [23]:
save_models(models_lbp,"lbp")

In [24]:
save_models(models_lbp2,"lbp")


In [25]:
save_models(models_lbp3,"lbp")


In [26]:
save_models(models_lbp4,"lbp")


In [27]:
save_models(models_lbp5,"lbp")
